<a href="https://colab.research.google.com/github/LorenzoBoccalon/N-queens-problem/blob/master/N_queens_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
"""https://arxiv.org/pdf/1802.02006.pdf"""
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
from typing import List, Tuple, Optional

# create global random number generator
RNG = np.random.default_rng(seed=42)

# type aliasing
Individual = List[int]
Pair = Tuple[Individual, Individual]
Point2D = Tuple[int, int]
Table = List[List[int]]


def genome_to_table(individual: Individual) -> Table:
    """transform genome to binary 2D-array"""
    nq = len(individual)
    tbl = np.zeros((nq, nq), np.int8)
    for r, c in enumerate(individual):
        tbl[r, c] = 1
    return tbl

def prettyprint(table: Table):
    """print a table in a nicer way"""
    nq = len(table)
    for i in range(nq):
        for j in range(nq):
            s = "Q" if table[i][j] else "-"
            print(s, end=" ")
        print()

def diag_attack(x: Point2D, y: Point2D) -> bool:
    """formula to check if coordinates intersect along a diagonal"""
    return abs(x[0] - y[0]) == abs(x[1] - y[1])


def count_attacks(genome: Individual) -> int:
    """given a chessboard disposition return the number of attacks.

    min = 0, no attacks
    max = NQ choose 2, i.e. every queen attack all the others"""
    cs = genome_to_coordinates(genome)
    tot = 0
    # generator of all pair combinations
    all_pairs = itertools.combinations(cs, 2)
    for c1, c2 in all_pairs:
        # same row
        if c1[0] == c2[0]:  
            tot += 1
        # same col
        if c1[1] == c2[1]:  
            tot += 1
        # same diag
        if diag_attack(c1, c2):  
            tot += 1
    return tot


def generate_genome(NQ: int = 8) -> Individual:
    """generate a random chessboard with NQ queens"""
    return RNG.permutation(NQ)


def genome_to_coordinates(genome: Individual) -> List[Point2D]:
    """given a genome return list of coordinates of the queens"""
    return np.array([(r, c) for r, c in enumerate(genome)])


def genetic_algo_solution(NQ: int = 8, NP: int = 100):
    """given NQ queens find a chessboard disposition where each queen does not attack any other. Use a genetic algorithm approach with a population of NP individuals per generation"""
    # set a limit
    max_iterations = 1_000_000
    solution = None
    pop = init_population(NQ, NP)
    for t in tqdm(range(max_iterations)):
        new_pop = []
        fs = fitness(pop)
        solution = check_population(pop, fs)
        if solution is not None:
            return solution
        ps = fitness_to_prob(fs)
        for _ in range(NP):
            (p1, p2) = selection(pop, ps)
            children = crossover((p1, p2)), crossover((p2, p1))
            children = [mutation(c) for c in children]
            new_pop = new_pop + children
        pop = new_pop


def init_population(NQ: int = 8, NP: int = 100) -> List[Individual]:
    """generate NP genomes of length NQ"""
    return [generate_genome(NQ) for _ in range(NP)]


def check_population(
    individuals: List[Individual], fitness: List[int]
) -> Optional[Individual]:
    """check if solution was found and eventualy return it"""
    for i, f in zip(individuals, fitness):
        if f == 0:
            return i
    return


def fitness(individuals: List[Individual]) -> List[int]:
    """given individuals return their fitness values"""
    # we use `count_attacks`, which is best if returns 0
    # therefore we minimise the fitness
    return [count_attacks(i) for i in individuals]


def fitness_to_prob(fitness: List[int]) -> List[float]:
    """transform list of fitness values into probabilities"""
    # numpy wrapping
    fitness = np.array(fitness)
    # add small quantity to prevent 0 div
    fitness = fitness + 1e-10
    # inverse function
    probs = 1 / fitness
    # normalise into probabilities'
    probs = probs / probs.sum()
    return probs.tolist()


def selection(individuals: List[Individual], probs: List[float]) -> Pair:
    """given individuals and probabilities of being chosen return a pair for reproduction"""
    p1, p2 = RNG.choice(individuals, size=2, replace=False, p=probs)
    return p1, p2


def crossover(parents: Pair, proportion: float = 0.5, debug: bool = False) -> Individual:
    """given parents pair generate a child"""
    p1, p2 = parents
    nq = len(p1)
    chunk_size = int(nq * proportion)
    split_point = RNG.choice(range(0, nq - chunk_size))
    start, end = split_point, split_point + chunk_size
    p1_genes = np.array(p1[start:end], np.int8)
    p2_genes = np.array([g for g in p2 if g not in p1_genes], np.int8)
    if(debug):
        print(f"split_point={split_point}, chunk_size={chunk_size}")
        print(f"p1_genes={p1_genes}, p2_genes={p2_genes}")
    child = np.concatenate((
        p2_genes[:start],
        p1_genes,
        p2_genes[start:],
    ))
    return child


def mutation_alternative(individual: Individual, prob: float = 1e-3) -> Individual:
    """mutate individual with small probability"""
    # toss unbalanced coin
    mutate_prob = prob
    dont_mutate_prob = 1 - prob
    if RNG.choice(2, p=[dont_mutate_prob, mutate_prob]):
        nq = len(individual)
        # select a gene (uniform prob)
        pos = RNG.choice(nq)
        # generate new gene
        old_gene = individual[pos]
        # sum and modulo trick to prevent the same gene
        new_gene = (old_gene + RNG.choice(range(1, nq))) % nq
        individual[pos] = new_gene
    return individual

def swap(individual: Individual) -> Individual:
    """swap two genes"""
    nq = len(individual)
    a, b = RNG.choice(nq, 2, replace=False)
    individual[a], individual[b] = individual[b], individual[a]
    return individual

def mutation(individual: Individual, prob: float = 1e-2) -> Individual:
    """mutate individual with small probability"""
    # toss unbalanced coin
    mutate_prob = prob
    dont_mutate_prob = 1 - prob
    if RNG.choice(2, p=[dont_mutate_prob, mutate_prob]):
        individual = swap(individual)
    return individual

def simulate_genetic_algo(NQ: int = 8):
    print("-----------------")
    print("Genetic Algorithm")
    start = time.perf_counter()
    solution = genetic_algo_solution(NQ=NQ)
    table = genome_to_table(solution)
    delta = time.perf_counter() - start
    print()
    print(f"Solution found in {delta} seconds!")
    prettyprint(table)
    print(f"Genome: {solution}")

# for simulated annealing we can reuse the type/nomenclature
def neighbor(individual: Individual, step: int = 3) -> Individual:
    """perturbate a point to generate a new one"""
    # swap `step` elements chosen randomly
    for _ in range(step):
        individual = swap(individual)
    return individual

def simulated_annealing_solution(NQ: int = 8, n_iterations: int = 1_000_000, step: int = 1, temp: float = 100.0):
    """given NQ queens find a chessboard disposition where each queen does not attack any other. Use simulated annealing search to find a minimum of the cost function, i.e. number of attacks"""
    # inspired by https://machinelearningmastery.com/
    # generate an initial point
    best = generate_genome(NQ)
    # evaluate the initial point
    best_eval = count_attacks(best)
    # current working solution
    curr, curr_eval = best, best_eval
    # run the algorithm
    print(f">start\tf({best}) = {best_eval}")
    for i in range(n_iterations):
        # take a step
        candidate = neighbor(curr, step)
        # evaluate candidate point
        candidate_eval = count_attacks(candidate)
        # check for new best solution
        if candidate_eval < best_eval:
            # store new best point
            best, best_eval = candidate, candidate_eval
            # report progress
            print(f">{i}\tf({best}) = {best_eval}")
        # difference between candidate and current point evaluation
        diff = candidate_eval - curr_eval
        # calculate temperature for current epoch
        t = temp / float(i + 1)
        # calculate metropolis acceptance criterion
        metropolis = np.exp(-diff / t)
        # check if we should keep the new point
        if diff < 0 or RNG.random() < metropolis:
            # store the new current point
            curr, curr_eval = candidate, candidate_eval
        # break early if solution is found
        if candidate_eval == 0: break
    return [best, best_eval]

def simulate_simulated_annealing_algo(NQ: int = 8):
    print("-------------------")
    print("Simulated Annealing")
    start = time.perf_counter()
    solution, minimum = simulated_annealing_solution(NQ=NQ)
    delta = time.perf_counter() - start
    if minimum == 0:
        table = genome_to_table(solution)
        print()
        print(f"Solution found in {delta} seconds!")
        prettyprint(table)
        print(f"Genome: {solution}")
    else:
        print("solution not found :(")

# test both algos!
for nq in [8, 10, 12, 14]:
    print(f"TEST for {nq} QUEENS")
    simulate_genetic_algo(nq)
    simulate_simulated_annealing_algo(nq)
    print("---------------------")

TEST for 8 QUEENS
-----------------
Genetic Algorithm


  0%|          | 1/1000000 [00:00<15:16:30, 18.18it/s]



Solution found in 0.06772730000011506 seconds!
- - - - - - Q - 
- - Q - - - - - 
- - - - - - - Q 
- Q - - - - - - 
- - - - Q - - - 
Q - - - - - - - 
- - - - - Q - - 
- - - Q - - - - 
Genome: [6 2 7 1 4 0 5 3]
-------------------
Simulated Annealing
>start	f([4 1 7 6 2 0 5 3]) = 1
>749	f([3 6 2 7 1 4 0 5]) = 0

Solution found in 0.1045663700001569 seconds!
- - - Q - - - - 
- - - - - - Q - 
- - Q - - - - - 
- - - - - - - Q 
- Q - - - - - - 
- - - - Q - - - 
Q - - - - - - - 
- - - - - Q - - 
Genome: [3 6 2 7 1 4 0 5]
---------------------
TEST for 10 QUEENS
-----------------
Genetic Algorithm


  0%|          | 14/1000000 [00:01<21:16:13, 13.06it/s]



Solution found in 1.1070338149997951 seconds!
- - Q - - - - - - - 
- - - - Q - - - - - 
- Q - - - - - - - - 
- - - - - - - - Q - 
- - - - - Q - - - - 
- - - - - - - - - Q 
- - - - - - Q - - - 
- - - Q - - - - - - 
Q - - - - - - - - - 
- - - - - - - Q - - 
Genome: [2 4 1 8 5 9 6 3 0 7]
-------------------
Simulated Annealing
>start	f([1 4 3 6 2 7 9 0 5 8]) = 5
>2	f([8 4 3 6 2 7 5 0 1 9]) = 4
>6	f([2 4 8 9 3 0 5 7 1 6]) = 3
>30	f([8 1 4 0 9 6 3 2 7 5]) = 2
>31	f([8 1 4 2 9 6 3 0 7 5]) = 1
>1433	f([4 1 7 9 6 2 0 8 3 5]) = 0

Solution found in 0.25448371099992073 seconds!
- - - - Q - - - - - 
- Q - - - - - - - - 
- - - - - - - Q - - 
- - - - - - - - - Q 
- - - - - - Q - - - 
- - Q - - - - - - - 
Q - - - - - - - - - 
- - - - - - - - Q - 
- - - Q - - - - - - 
- - - - - Q - - - - 
Genome: [4 1 7 9 6 2 0 8 3 5]
---------------------
TEST for 12 QUEENS
-----------------
Genetic Algorithm


  0%|          | 166/1000000 [00:11<19:48:24, 14.02it/s]



Solution found in 11.86374532200034 seconds!
- - - - - - Q - - - - - 
- - - - - - - - - - - Q 
- - - - - Q - - - - - - 
- - Q - - - - - - - - - 
Q - - - - - - - - - - - 
- - - - - - - - - - Q - 
- - - - - - - Q - - - - 
- - - - - - - - - Q - - 
- - - Q - - - - - - - - 
- Q - - - - - - - - - - 
- - - - Q - - - - - - - 
- - - - - - - - Q - - - 
Genome: [ 6 11  5  2  0 10  7  9  3  1  4  8]
-------------------
Simulated Annealing
>start	f([ 4 10  2  1  5  6  9 11  0  7  8  3]) = 11
>0	f([ 4 10  2  1  5  6  9  0 11  7  8  3]) = 9
>2	f([ 4 10  2  1  9  6  3  0 11  7  8  5]) = 8
>9	f([ 3 10  9  1  4  6 11  0  5  7  8  2]) = 7
>14	f([ 6 10  9  7  4 11  8  2  5  1  3  0]) = 6
>15	f([ 6 10  9  7  0 11  8  2  5  1  3  4]) = 5
>24	f([ 5 11  8  7  0  1  4  2  9  6  3 10]) = 4
>92	f([ 9  3  8  2  1  6  0  5 11  7 10  4]) = 2
>689	f([ 5  8  6  1  2 11  7 10  4  9  0  3]) = 1
>7065	f([ 2  4 11  9  5 10  0  7  3  1  6  8]) = 0

Solution found in 1.4969216169997708 seconds!
- - Q - - - - - - - - - 
- 

  0%|          | 576/1000000 [00:46<22:18:36, 12.44it/s]



Solution found in 46.30265993099965 seconds!
- - - - - - - - - - - - Q - 
- - - - - - - - - Q - - - - 
- - - - Q - - - - - - - - - 
- - - - - - - - Q - - - - - 
Q - - - - - - - - - - - - - 
- - - - - - - - - - - Q - - 
- - - Q - - - - - - - - - - 
- Q - - - - - - - - - - - - 
- - - - - - - Q - - - - - - 
- - - - - - - - - - - - - Q 
- - - - - - - - - - Q - - - 
- - - - - - Q - - - - - - - 
- - Q - - - - - - - - - - - 
- - - - - Q - - - - - - - - 
Genome: [12  9  4  8  0 11  3  1  7 13 10  6  2  5]
-------------------
Simulated Annealing
>start	f([ 4  7  9 13 12  3  6  1 10  5  2 11  8  0]) = 7
>1	f([ 4  7  9 13  1  3  6  5 10 12  2 11  8  0]) = 6
>2	f([ 4  7  9 13  1  3  5  6 10 12  2 11  8  0]) = 4
>55	f([10  1  9  2  3 13  7  5 11  0 12  6  8  4]) = 3
>1715	f([ 5  7  9  4 12  8  3 11  2 10  1  6  0 13]) = 2
>16421	f([ 6  8  0 11  1 10  5  3  9  4  2 13 12  7]) = 1
>395857	f([ 2 10  7  3 11  0 12  1  5  8 13  9  4  6]) = 0

Solution found in 94.36803361500006 seconds!
- - Q - - - - -